In [2]:
%matplotlib inline

# 신경망 모델 구성하기

신경망은 데이터에 대한 연산을 수행하는 계층(layer)/모듈(module)로 구성되어 있다.
[`torch.nn`](<https://pytorch.org/docs/stable/nn.html>) 네임스페이스는 신경망을 구성하는데 필요한 모든 구성 요소를 제공한다.
PyTorch의 모든 모듈은 [`nn.Module`](<https://pytorch.org/docs/stable/generated/torch.nn.Module.html>)의 하위 클래스(subclass)
이다. 신경망은 그 자체로 다른 모듈 혹은 계층(layer)들로 구성된 하나의 모듈이다. 이러한 중첩된 구조는 복잡한 아키텍처를 쉽게 구축하고 관리할 수 있게 한다.

이제 FashionMNIST 데이터셋의 이미지들을 분류하는 신경망을 구성해보자.


In [3]:
import os
import torch
from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets, transforms

학습을 위한 장치 얻기
------------------------------------------------------------------------------------------

가능하다면 GPU와 같은 하드웨어 가속기에서 모델을 학습하는 것이 효율적이다.
[`torch.cuda`](<https://pytorch.org/docs/stable/notes/cuda.html>)를 사용할 수 있는지
확인하고 그렇지 않으면 CPU를 계속 사용한다.

In [4]:
device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"Using {device} device")

Using cuda device


클래스 정의하기
------------------------------------------------------------------------------------------

신경망 모델을 ``nn.Module``의 하위클래스로 정의하고, ``__init__``에서 신경망 계층들을 초기화한다.
``nn.Module``을 상속받은 모든 클래스는 ``forward`` 메소드에 입력 데이터에 대한 연산들을 구현한다.



In [5]:
class NeuralNetwork(nn.Module):
    def __init__(self):
        super(NeuralNetwork, self).__init__()
        self.flatten = nn.Flatten()
        self.linear_relu_stack = nn.Sequential(
            nn.Linear(28*28, 512),
            nn.ReLU(),
            nn.Linear(512, 512),
            nn.ReLU(),
            nn.Linear(512, 10),
        )

    def forward(self, x):
        x = self.flatten(x)
        logits = self.linear_relu_stack(x)
        return logits

``NeuralNetwork``의 인스턴스(instance)를 생성하고 이를 ``device``로 이동한 뒤,
구조(structure)를 출력한다.



In [6]:
model = NeuralNetwork().to(device)
print(model)

NeuralNetwork(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (linear_relu_stack): Sequential(
    (0): Linear(in_features=784, out_features=512, bias=True)
    (1): ReLU()
    (2): Linear(in_features=512, out_features=512, bias=True)
    (3): ReLU()
    (4): Linear(in_features=512, out_features=10, bias=True)
  )
)


모델에 입력 데이터를 전달하면 약간의
[백그라운드 연산들](<https://github.com/pytorch/pytorch/blob/270111b7b611d174967ed204776985cefca9c144/torch/nn/modules/module.py#L866>)과 함께
모델의 ``forward`` 메서드가 자동으로 실행된다. ``model.forward()``를 사용자가 직접 호출하지는 않는다.

입력 데이터에 대해 모델을 실행하면 각 분류(class)에 대한 원시(raw) 예측값이 저장된 10-차원 텐서가 반환된다. 원시 예측값을 ``nn.Softmax`` 모듈의 인스턴스에 통과시켜 예측 확률을 얻는다.



In [7]:
X = torch.rand(1, 28, 28, device=device) # should actually be 1, 1, 28, 28 but torch understands this fine
logits = model(X)
pred_probab = nn.Softmax(dim=1)(logits) # not strictly necessary here
# softmax: maps each inputs to [0,1] such that inputs' size relation is preserved and sum of them is 1
y_pred = pred_probab.argmax(1)
print(f"Predicted class: {y_pred}")

Predicted class: tensor([5], device='cuda:0')


------------------------------------------------------------------------------------------




모델 계층(Layer)
------------------------------------------------------------------------------------------

FashionMNIST 모델의 계층들을 살펴보자. 이를 설명하기 위해, 28x28 크기의 랜덤 이미지 3개로 구성된
미니배치를 가져와, 신경망을 통과할 때 어떤 일이 발생하는지 알아본다.



In [10]:
input_image = torch.rand(3,28,28)
print(input_image)
print(input_image.size())

tensor([[[[9.9928e-01, 2.0988e-01, 5.9473e-01,  ..., 8.8738e-01,
           7.3325e-01, 4.1946e-01],
          [8.9353e-01, 6.7821e-01, 2.7026e-01,  ..., 4.5129e-01,
           1.0666e-01, 3.8061e-01],
          [2.9219e-01, 4.9672e-01, 3.3059e-01,  ..., 5.8252e-01,
           1.1636e-01, 6.8737e-01],
          ...,
          [3.1693e-01, 2.3180e-01, 6.1635e-03,  ..., 7.0889e-01,
           5.9139e-01, 4.3429e-01],
          [9.7392e-01, 4.8667e-01, 1.1410e-01,  ..., 8.4743e-01,
           4.3024e-01, 6.8225e-01],
          [5.6175e-01, 9.0237e-01, 5.5603e-01,  ..., 3.5364e-01,
           2.0274e-01, 3.9082e-01]],

         [[6.8753e-01, 8.5767e-01, 4.3355e-01,  ..., 3.3315e-01,
           2.6087e-01, 5.7910e-01],
          [6.1294e-01, 8.2156e-01, 1.3153e-01,  ..., 4.7208e-01,
           8.9910e-01, 5.9171e-02],
          [9.3229e-01, 6.2257e-01, 4.7749e-01,  ..., 7.0579e-01,
           7.1962e-01, 8.6955e-01],
          ...,
          [6.5623e-01, 4.5831e-03, 7.5773e-01,  ..., 2.7293

#### `nn.Flatten`

[`nn.Flatten`](<https://pytorch.org/docs/stable/generated/torch.nn.Flatten.html>) 계층은
각 28x28의 2D 이미지를 784 픽셀 값을 갖는 1차원 배열로 변환한다. dim=0의 미니배치 차원은 유지된다.



In [11]:
flatten = nn.Flatten()
flat_image = flatten(input_image)
print(flat_image.size()) # First dimension is treated as batch

torch.Size([3, 2352])


#### `nn.Linear`

[선형 계층](<https://pytorch.org/docs/stable/generated/torch.nn.Linear.html>)은 저장된 가중치(weight)와
편향(bias)을 사용하여 입력에 선형 변환(linear transformation)을 적용하는 모듈이다.

In [ ]:
layer1 = nn.Linear(in_features=28*28, out_features=20)
hidden1 = layer1(flat_image)
print(hidden1.size())
print(hidden1)

torch.Size([3, 20])
tensor([[ 0.0723,  0.2944,  0.0688, -0.0080, -0.1988, -0.1473,  0.0402, -0.1625,
          0.0051,  0.3092, -0.2408, -0.0197, -0.4539, -0.6212, -0.3582, -0.2164,
         -0.1515,  0.2352, -0.6302,  0.4274],
        [-0.1705,  0.1423,  0.0308, -0.5483, -0.1650, -0.0633, -0.4033, -0.1392,
          0.4359,  0.2693, -0.1483, -0.0269, -0.4287, -0.5644, -0.3774, -0.2523,
          0.2513, -0.0018, -0.3849,  0.3450],
        [-0.0281,  0.2736,  0.0202, -0.1368, -0.2337, -0.2621, -0.2145, -0.4218,
          0.3157,  0.4130, -0.2214,  0.2861, -0.3192, -0.4156, -0.4520,  0.1149,
         -0.2612,  0.0755, -0.5630,  0.6957]], grad_fn=<AddmmBackward0>)


#### `nn.ReLU`

비선형 활성화(activation)는 모델의 입력과 출력 사이에 비선형적인 복잡한 관계(mapping)를 만든다.
비선형 활성화는 선형 변환 후에 적용되어 *비선형성(nonlinearity)* 을 도입하고, 신경망이 다양한 현상을 학습할 수 있도록 돕는다.

이 모델에서는 [`nn.ReLU`](<https://pytorch.org/docs/stable/generated/torch.nn.ReLU.html>)를 사용하지만, 다양한 다른 활성화 함수를 사용할 수도 있다. 

**Note:** $ReLu(x) = \max(0, x)$이다.



In [ ]:
print(f"Before ReLU: {hidden1}\n\n")
hidden1 = nn.ReLU()(hidden1)
print(f"After ReLU: {hidden1}")

Before ReLU: tensor([[ 0.0723,  0.2944,  0.0688, -0.0080, -0.1988, -0.1473,  0.0402, -0.1625,
          0.0051,  0.3092, -0.2408, -0.0197, -0.4539, -0.6212, -0.3582, -0.2164,
         -0.1515,  0.2352, -0.6302,  0.4274],
        [-0.1705,  0.1423,  0.0308, -0.5483, -0.1650, -0.0633, -0.4033, -0.1392,
          0.4359,  0.2693, -0.1483, -0.0269, -0.4287, -0.5644, -0.3774, -0.2523,
          0.2513, -0.0018, -0.3849,  0.3450],
        [-0.0281,  0.2736,  0.0202, -0.1368, -0.2337, -0.2621, -0.2145, -0.4218,
          0.3157,  0.4130, -0.2214,  0.2861, -0.3192, -0.4156, -0.4520,  0.1149,
         -0.2612,  0.0755, -0.5630,  0.6957]], grad_fn=<AddmmBackward0>)


After ReLU: tensor([[0.0723, 0.2944, 0.0688, 0.0000, 0.0000, 0.0000, 0.0402, 0.0000, 0.0051,
         0.3092, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.2352,
         0.0000, 0.4274],
        [0.0000, 0.1423, 0.0308, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.4359,
         0.2693, 0.0000, 0.0000, 0.0000, 0.0000, 0.00

#### `nn.Sequential`

[`nn.Sequential`](<https://pytorch.org/docs/stable/generated/torch.nn.Sequential.html>)은 순서를 갖는
모듈의 컨테이너이다. 데이터는 모듈들을 나열된 순서대로 통과한다. 순차 컨테이너(sequential container)를 사용하여 아래의 ``seq_modules``와 같은 신경망을 빠르게 만들 수 있다.



In [ ]:
seq_modules = nn.Sequential(
    flatten,
    layer1,
    nn.ReLU(),
    nn.Linear(20, 10)
)
input_image = torch.rand(3,28,28)
logits = seq_modules(input_image)

#### `nn.Softmax`

신경망의 마지막 선형 계층은 [`nn.Softmax`](<https://pytorch.org/docs/stable/generated/torch.nn.Softmax.html>) 모듈에 전달될
[-\infty, \infty] 범위의 원시 값(raw value)인 `logits`를 반환한다. `nn.Softmax` 계층은 logits는 모델의 각 분류(class)에 대한 예측 확률을 나타내도록
[0, 1] 범위로 비례하여 조정(scale)한다. ``dim`` 매개변수는 값의 합이 1이 되는 차원을 나타낸다.



In [ ]:
softmax = nn.Softmax(dim=1)
pred_probab = softmax(logits)

**Note:** `SoftMax`함수 $\sigma : R^K \longrightarrow (0, 1)^K$의 정의: $\sigma(z)_i = \frac{e^{z_i}}{\sum_{j=1}^K e^{z_j}}$

모델 매개변수
------------------------------------------------------------------------------------------

신경망에서 많은 계층들은 *매개변수화(parameterize)*되어 있다. 즉, 학습 중에 최적화되는 가중치(weight)와 편향(bias)을 가진다. ``nn.Module``을 상속하면 모델 객체 내부의 모든 필드들이 자동으로 추적(track)되며, 모델의 ``parameters()`` 및
``named_parameters()`` 메소드로 모든 매개변수에 접근할 수 있게 된다.

이 예제에서는 각 매개변수들을 순회하며(iterate), 매개변수의 크기와 값을 출력한다.




In [ ]:
print(f"Model structure: {model}\n\n")

for name, param in model.named_parameters():
    print(f"Layer: {name} | Size: {param.size()} | Values : {param[:2]} \n")

Model structure: NeuralNetwork(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (linear_relu_stack): Sequential(
    (0): Linear(in_features=784, out_features=512, bias=True)
    (1): ReLU()
    (2): Linear(in_features=512, out_features=512, bias=True)
    (3): ReLU()
    (4): Linear(in_features=512, out_features=10, bias=True)
  )
)


Layer: linear_relu_stack.0.weight | Size: torch.Size([512, 784]) | Values : tensor([[-0.0084, -0.0223, -0.0094,  ...,  0.0350, -0.0234, -0.0309],
        [ 0.0003, -0.0201, -0.0074,  ..., -0.0274, -0.0022,  0.0242]],
       device='cuda:0', grad_fn=<SliceBackward0>) 

Layer: linear_relu_stack.0.bias | Size: torch.Size([512]) | Values : tensor([-0.0234,  0.0290], device='cuda:0', grad_fn=<SliceBackward0>) 

Layer: linear_relu_stack.2.weight | Size: torch.Size([512, 512]) | Values : tensor([[ 0.0291, -0.0322,  0.0107,  ...,  0.0326,  0.0043, -0.0377],
        [-0.0336,  0.0261,  0.0130,  ...,  0.0281,  0.0441,  0.0378]],
       device='cuda:0', grad_fn=<Sl

------------------------------------------------------------------------------------------




더 읽어보기
------------------------------------------------------------------------------------------
- [`torch.nn API`](<https://pytorch.org/docs/stable/nn.html>)

